### Imports

In [37]:
from sage.all import *
from sage.matrix.berlekamp_massey import berlekamp_massey
import time

### BlackBox Class

In [38]:
class BlackBox: 
    def __init__(self, matrix): 
        self.__matrix = matrix.sparse_matrix()
    def prod(self, vector):
        return self.__matrix * vector

### Matrix, Vector & Linear System Generators

In [39]:
# Generate random square & invertible matrix with given field, dimension, sparsity
def generate_nonsingular_square_matrix(field, dim, sparsity):
    invertible = False
    attempts = 0
    max_matrix_attempts = 1000 # maximum 1000 attemmpts

    # Keep generating new matrices until get an invertible one
    while not invertible and attempts < max_matrix_attempts: 
        attempts += 1
        A = random_matrix(field, dim, dim, density=(1 - sparsity), sparse=True)
        invertible = A.is_invertible()
    if not invertible:
        raise RuntimeError(f"Failed to generate invertible matrix after {max_matrix_attempts} attempts")
    return A

# Generate random vector with given field and dimension
def generate_vector(field, dim):
    return vector(field, [field.random_element() for _ in range(dim)])

# Generate random linear system (Ax = b) with invertible square A and b
def generate_linear_system(field=None, dim=None, sparsity=0.9):
    if not dim: 
        dim = ZZ.random_element(100, 300)
    if not field:
        field = GF(1009)
    A = generate_nonsingular_square_matrix(field, dim, sparsity)
    b = generate_vector(field, dim)
    return A, b, dim, field

### Krylov subspace generator

In [40]:
def krylov(black_box, b, order, u=None):
    krylov_seq = []
    last_element = b
    for _ in range(order):
        if u == None: 
            krylov_seq.append(last_element) # {A^ib} sequence
        else:
            krylov_seq.append(u.dot_product(last_element)) # {u * A^ib} sequence
        last_element = black_box.prod(last_element)
    return krylov_seq

### Horner's

In [41]:
# Horner's method to recursively compute poly
# c(x) = (..((c_n)x + c_{n-1})x + ... + c_1)x + c_0 
# This horner function computes c(A)b
def horner(coeffs, black_box, vector):
    # No coeffs: return 0
    if not coeffs:
        return vector * 0
    # If only 1 coeff: return c_0*b
    if len(coeffs) == 1:
        return coeffs[0] * vector
    # Recursive call
    return black_box.prod(horner(coeffs[1:], black_box, vector)) + coeffs[0] * vector

### Wiedemann's Algorithm

In [42]:
# Implementation of Wiedemann's Algorithm on nonsingular matrices
def wiedemann(black_box, b, dim, field): 
    R = PolynomialRing(field, 'x') # Define Poly Ring

    max_attempts = 10 # Avoid forever loop
    attempt = 0
    base_poly = R(1) # Polynomial to store factors of failed min poly

    # If RHS is zero, return the zero vector 
    if b.is_zero():
        print("zero RHS")
        return vector(field, [0] * dim), 1, base_poly

    # Main loop to find solution that satisfies Ax = b
    while attempt <= max_attempts:
        attempt += 1
        u = generate_vector(field, dim) # Random vector u

        # Regenerate u if u is zero
        while u.is_zero():
            u = generate_vector(field, dim)

        # Generate Krylov sequence & dot product with u
        krylov_sequence = krylov(black_box, b, 2 * dim, u)

        # Generate min poly of {u^TA^ib}
        m_poly = berlekamp_massey(krylov_sequence)

        # Update base_poly with factors from m_poly
        factors_m = dict(m_poly.factor())
        factors_base = dict(base_poly.factor())
        
        # For each factor in m_poly, update base_poly if higher multiplicity
        for factor, mult_m in factors_m.items():
            mult_base = factors_base.get(factor, 0)
            if mult_m > mult_base: 
                base_poly *= factor**(mult_m - mult_base)
        
        # Check if base_poly has degree equal to matrix dimension
        base_poly_coeffs = list(base_poly)
        try:
                h_coeffs = [-c / base_poly_coeffs[0] for c in base_poly_coeffs[1:]] # h(x)
                result = horner(h_coeffs, black_box, b) # x = h(A)b
                # Verify solution by checking Ax - b = 0
                if (black_box.prod(result) - b).is_zero(): 
                    print(f"Valid solution: attempt #{attempt}")
                    return result, attempt, base_poly
                else:
                    print(f"Attempt {attempt} failed...")
        except Exception as e:
                print(f"Attempt {attempt} failed: {e}")
    # Failed after max attempts
    print("Max attempts failed")
    raise RuntimeError("Wiedemann failed after max attempts")

### Timed Test Function

In [43]:
def wiedemann_timed_test(A, b, dim, field):
    bbox_A = BlackBox(A)
    start = time.time()
    x, attempts, w_minpoly = wiedemann(bbox_A, b, dim, field)
    end = time.time()
    print(f"Finished Wiedemann in {end - start:.3f}s ({attempts} attempt{"s" if attempts != 1 else ""})")
    start_sage = time.time()
    assert x == A.solve_right(b), f"Failed: Solution different from Sage built-in solver"
    end_sage = time.time()
    print(f'Solution verified against Sage built-in solver ({end_sage - start_sage:.3f}s)')
    assert w_minpoly == A.minpoly(), 'Wiedemann generated min poly is different from actual minpoly'
    print('Wiedemann generated min poly is consistent with actual min poly')
    return attempts

### Random Test

In [44]:
def test_random_matrices(num_tests):
    i = 0
    failed = 0
    while i < num_tests:
        i += 1
        try:
            print('---')
            print(f'Case {i}...', end=' ')
            sparsity = 0.85 + (0.95 - 0.85) * random() # Random sparsity in [0.85, 0.95]
            A, b, dim, field = generate_linear_system(sparsity = sparsity)
            wiedemann_timed_test(A, b, dim, field)
        except Exception as e:
            print(f'Failed: {e}')
            failed += 1
    print('===')
    print(f'Failed {failed} cases')

In [45]:
test_random_matrices(5)

---
Case 1... Valid solution: attempt #1
Finished Wiedemann in 2.797s (1 attempt)
Solution verified against Sage built-in solver (0.033s)
Wiedemann generated min poly is consistent with actual min poly
---
Case 2... Valid solution: attempt #1
Finished Wiedemann in 2.262s (1 attempt)
Solution verified against Sage built-in solver (0.031s)
Wiedemann generated min poly is consistent with actual min poly
---
Case 3... Valid solution: attempt #1
Finished Wiedemann in 0.404s (1 attempt)
Solution verified against Sage built-in solver (0.006s)
Wiedemann generated min poly is consistent with actual min poly
---
Case 4... Valid solution: attempt #1
Finished Wiedemann in 2.373s (1 attempt)
Solution verified against Sage built-in solver (0.021s)
Wiedemann generated min poly is consistent with actual min poly
---
Case 5... Valid solution: attempt #1
Finished Wiedemann in 0.746s (1 attempt)
Solution verified against Sage built-in solver (0.009s)
Wiedemann generated min poly is consistent with actual